# Machine Learning - Unsupervised Learning and Feature Engineering
## Unit 6. Automated Feature Generation
### Lecturer: Tai Le Quy
#### 31.8.2023
Thanks Prof. Dr. Christian Müller-Kett for his code.

## 1 Automated Feature Generation

### 1.1 Featuretools

In [ ]:
#Install the featuretools from the command prompt: version 1.11.1 (30.8.2023)
pip install featuretools

In [1]:
#%% import libraries
import pandas as pd
import featuretools as ft

C:\ProgramData\Anaconda3\lib\site-packages\woodwork\__init__.py:23: FutureWarning: Woodwork may not support Python 3.7 in next non-bugfix release.
  "Woodwork may not support Python 3.7 in next non-bugfix release.", FutureWarning
C:\ProgramData\Anaconda3\lib\site-packages\featuretools\__init__.py:67: FutureWarning: Featuretools may not support Python 3.7 in next non-bugfix release.
  FutureWarning,


In [2]:
#%% Remove any limit on the number of columns to display 
pd.options.display.max_columns = None 

In [3]:
#%% Remove any limit on the number of rows to display
pd.options.display.max_rows = None

In [4]:
#%% Display the list of primitives
print(ft.list_primitives())

                                name         type  dask_compatible  \
0                               last  aggregation            False   
1                         num_unique  aggregation             True   
2                                sum  aggregation             True   
3                      n_most_common  aggregation            False   
4                                all  aggregation             True   
5                   time_since_first  aggregation            False   
6                                std  aggregation             True   
7                               skew  aggregation            False   
8                            entropy  aggregation            False   
9                              count  aggregation             True   
10                          num_true  aggregation             True   
11                               max  aggregation             True   
12                              mode  aggregation            False   
13                  

### 1.2 Automatically generate new features

In [2]:
import featuretools as ft
import pandas as pd

C:\ProgramData\Anaconda3\lib\site-packages\woodwork\__init__.py:23: FutureWarning: Woodwork may not support Python 3.7 in next non-bugfix release.
  "Woodwork may not support Python 3.7 in next non-bugfix release.", FutureWarning
C:\ProgramData\Anaconda3\lib\site-packages\featuretools\__init__.py:67: FutureWarning: Featuretools may not support Python 3.7 in next non-bugfix release.
  FutureWarning,


In [3]:
#%% create sample data
Customers = pd.DataFrame({ 
    'C_ID': ['C1', 'C2'], 
    'Name': ['Martin', 'Julia'], 
    'Creation_date': ['2018-08-15', '2020-05-05']}, 
        columns = ['C_ID','Name','Creation_date'])
Orders = pd.DataFrame({ 
    'Ord_ID': ['1', '2', '3', '4', '5'], 
    'C_ID': ['C1', 'C2', 'C1', 'C1','C2']}, 
        columns = ['Ord_ID','C_ID'])
Payments = pd.DataFrame({ 
    'Ord_ID':['1', '5', '3', '4', '2'], 
    'Price':[500, 200, 300, 100, 900]}, 
        columns = ['Ord_ID', 'Price'])

In [4]:
Customers

,C_ID,Name,Creation_date
0,C1,Martin,2018-08-15
1,C2,Julia,2020-05-05


In [5]:
Orders

,Ord_ID,C_ID
0,1,C1
1,2,C2
2,3,C1
3,4,C1
4,5,C2


In [6]:
Payments

,Ord_ID,Price
0,1,500
1,5,200
2,3,300
3,4,100
4,2,900


In [7]:
#%% create 'customer' entitysets
es = ft.EntitySet(id = 'Customers')
es = es.add_dataframe( \
    dataframe_name = 'Customers', \
    dataframe = Customers, \
    index = 'C_ID', time_index = 'Creation_date')

In [8]:
es

Entityset: Customers
  DataFrames:
    Customers [Rows: 2, Columns: 3]
  Relationships:
    No relationships

In [9]:
#%% create orders entityset
es = es.add_dataframe( \
    dataframe_name = 'Orders', \
    dataframe = Orders, \
    index = 'Ord_ID')

In [10]:
#%% create payments entityset
es = es.add_dataframe( \
    dataframe_name = 'Payments', \
    dataframe = Payments, 
    make_index = True,
    index = 'P_ID')

In [13]:
es['Payments']

,P_ID,Ord_ID,Price
0,0,1,500
1,1,5,200
2,2,3,300
3,3,4,100
4,4,2,900


In [14]:
es

Entityset: Customers
  DataFrames:
    Customers [Rows: 2, Columns: 3]
    Orders [Rows: 5, Columns: 2]
    Payments [Rows: 5, Columns: 3]
  Relationships:
    No relationships

In [15]:
#%%
# Define the relationship between the parent 'Customers' 
# and the child 'Orders' linked together by 'C_ID'
r_Cust_Ord = ft.Relationship(entityset=es,parent_dataframe_name= 'Customers', parent_column_name='C_ID', 
                             child_dataframe_name= 'Orders',child_column_name='C_ID')

In [16]:
r_Cust_Ord

<Relationship: Orders.C_ID -> Customers.C_ID>

In [17]:
#%% Add the relationship to the entity set
es = es.add_relationship(relationship=r_Cust_Ord)

In [18]:
es

Entityset: Customers
  DataFrames:
    Customers [Rows: 2, Columns: 3]
    Orders [Rows: 5, Columns: 2]
    Payments [Rows: 5, Columns: 3]
  Relationships:
    Orders.C_ID -> Customers.C_ID

In [19]:
#%% define relationship between 'Orders' 
# and 'Payments'
r_Orders_Payments = ft.Relationship(entityset=es,parent_dataframe_name= 'Orders', 
                                    parent_column_name='Ord_ID',child_dataframe_name= 'Payments',
                                    child_column_name='Ord_ID')

In [20]:
#%% Add the relationship to the entity set
es = es.add_relationship(relationship=r_Orders_Payments)

In [21]:
#%% show entityset
es

Entityset: Customers
  DataFrames:
    Customers [Rows: 2, Columns: 3]
    Orders [Rows: 5, Columns: 2]
    Payments [Rows: 5, Columns: 3]
  Relationships:
    Orders.C_ID -> Customers.C_ID
    Payments.Ord_ID -> Orders.Ord_ID

In [22]:
#%% show aggregation primitives
primitives = ft.list_primitives()
pd.options.display.max_colwidth = 160
primitives[primitives['type']=="aggregation"].head(15)

,name,type,dask_compatible,spark_compatible,description,valid_inputs,return_type
0,all,aggregation,True,False,Calculates if all values are 'True' in a list.,"<ColumnSchema (Logical Type = Boolean)>, <ColumnSchema (Logical Type = BooleanNullable)>",<ColumnSchema (Logical Type = Boolean)>
1,mode,aggregation,False,False,Determines the most commonly repeated value.,<ColumnSchema (Semantic Tags = ['category'])>,None
2,trend,aggregation,False,False,Calculates the trend of a column over time.,"<ColumnSchema (Semantic Tags = ['numeric'])>, <ColumnSchema (Logical Type = Datetime) (Semantic Tags = ['time_index'])>",<ColumnSchema (Semantic Tags = ['numeric'])>
3,std,aggregation,True,True,"Computes the dispersion relative to the mean value, ignoring `NaN`.",<ColumnSchema (Semantic Tags = ['numeric'])>,<ColumnSchema (Semantic Tags = ['numeric'])>
4,entropy,aggregation,False,False,Calculates the entropy for a categorical column,<ColumnSchema (Semantic Tags = ['category'])>,<ColumnSchema (Semantic Tags = ['numeric'])>
5,time_since_last,aggregation,False,False,Calculates the time elapsed since the last datetime (default in seconds).,<ColumnSchema (Logical Type = Datetime) (Semantic Tags = ['time_index'])>,<ColumnSchema (Logical Type = Double) (Semantic Tags = ['numeric'])>
6,median,aggregation,False,False,Determines the middlemost number in a list of values.,<ColumnSchema (Semantic Tags = ['numeric'])>,<ColumnSchema (Semantic Tags = ['numeric'])>
7,avg_time_between,aggregation,False,False,Computes the average number of seconds between consecutive events.,<ColumnSchema (Logical Type = Datetime) (Semantic Tags = ['time_index'])>,<ColumnSchema (Logical Type = Double) (Semantic Tags = ['numeric'])>
8,n_most_common,aggregation,False,False,Determines the `n` most common elements.,<ColumnSchema (Semantic Tags = ['category'])>,None
9,percent_true,aggregation,True,False,Determines the percent of `True` values.,"<ColumnSchema (Logical Type = Boolean)>, <ColumnSchema (Logical Type = BooleanNullable)>",<ColumnSchema (Logical Type = Double) (Semantic Tags = ['numeric'])>


In [61]:
#%% show transformation primitives
primitives[primitives['type']=="transform"].head(15)

,name,type,dask_compatible,spark_compatible,description,valid_inputs,return_type
22,part_of_day,transform,True,True,Determines the part of day of a datetime.,<ColumnSchema (Logical Type = Datetime)>,<ColumnSchema (Logical Type = Categorical) (Semantic Tags = ['category'])>
23,rolling_min,transform,False,False,Determines the minimum of entries over a given window.,"<ColumnSchema (Semantic Tags = ['numeric'])>, <ColumnSchema (Logical Type = Datetime) (Semantic Tags = ['time_index'])>",<ColumnSchema (Logical Type = Double) (Semantic Tags = ['numeric'])>
24,add_numeric_scalar,transform,True,True,Add a scalar to each value in the list.,<ColumnSchema (Semantic Tags = ['numeric'])>,<ColumnSchema (Semantic Tags = ['numeric'])>
25,rolling_std,transform,False,False,Calculates the standard deviation of entries over a given window.,"<ColumnSchema (Semantic Tags = ['numeric'])>, <ColumnSchema (Logical Type = Datetime) (Semantic Tags = ['time_index'])>",<ColumnSchema (Logical Type = Double) (Semantic Tags = ['numeric'])>
26,num_characters,transform,True,True,Calculates the number of characters in a string.,<ColumnSchema (Logical Type = NaturalLanguage)>,<ColumnSchema (Semantic Tags = ['numeric'])>
27,and,transform,True,True,Element-wise logical AND of two lists.,"<ColumnSchema (Logical Type = BooleanNullable)>, <ColumnSchema (Logical Type = Boolean)>",<ColumnSchema (Logical Type = BooleanNullable)>
28,isin,transform,True,True,Determines whether a value is present in a provided list.,<ColumnSchema>,<ColumnSchema (Logical Type = Boolean)>
29,age,transform,True,False,Calculates the age in years as a floating point number given a,<ColumnSchema (Logical Type = Datetime) (Semantic Tags = ['date_of_birth'])>,<ColumnSchema (Logical Type = AgeFractional) (Semantic Tags = ['numeric'])>
30,is_month_end,transform,True,True,Determines the is_month_end attribute of a datetime column.,<ColumnSchema (Logical Type = Datetime)>,<ColumnSchema (Logical Type = BooleanNullable)>
31,is_year_end,transform,True,True,Determines if a date falls on the end of a year.,<ColumnSchema (Logical Type = Datetime)>,<ColumnSchema (Logical Type = BooleanNullable)>


In [24]:
#%% generate features
features, feature_names = ft.dfs( 
    entityset=es, 
    target_dataframe_name='Customers', 
    agg_primitives=['sum'], 
    trans_primitives=['year'])

In [25]:
features

,SUM(Payments.Price),YEAR(Creation_date)
C_ID,,
C1,900.0,2018
C2,1100.0,2020


In [27]:
#%% generate features
feats, feat_names = ft.dfs( entityset=es, 
    target_dataframe_name='Customers', 
    max_depth = 2)

In [28]:
feats

,COUNT(Orders),COUNT(Payments),MAX(Payments.Price),MEAN(Payments.Price),MIN(Payments.Price),SKEW(Payments.Price),STD(Payments.Price),SUM(Payments.Price),DAY(Creation_date),MONTH(Creation_date),...,STD(Orders.MAX(Payments.Price)),STD(Orders.MEAN(Payments.Price)),STD(Orders.MIN(Payments.Price)),STD(Orders.SKEW(Payments.Price)),STD(Orders.SUM(Payments.Price)),SUM(Orders.MAX(Payments.Price)),SUM(Orders.MEAN(Payments.Price)),SUM(Orders.MIN(Payments.Price)),SUM(Orders.SKEW(Payments.Price)),SUM(Orders.STD(Payments.Price))
C_ID,,,,,,,,,,,,,,,,,,,,,
C1,3,3,500.0,300.0,100.0,0.0,200.000000,900.0,15,8,...,200.000000,200.000000,200.000000,NaN,200.000000,900.0,900.0,900.0,0.0,0.0
C2,2,2,900.0,550.0,200.0,NaN,494.974747,1100.0,5,5,...,494.974747,494.974747,494.974747,NaN,494.974747,1100.0,1100.0,1100.0,0.0,0.0


In [29]:
feat_names

[<Feature: COUNT(Orders)>,
 <Feature: COUNT(Payments)>,
 <Feature: MAX(Payments.Price)>,
 <Feature: MEAN(Payments.Price)>,
 <Feature: MIN(Payments.Price)>,
 <Feature: SKEW(Payments.Price)>,
 <Feature: STD(Payments.Price)>,
 <Feature: SUM(Payments.Price)>,
 <Feature: DAY(Creation_date)>,
 <Feature: MONTH(Creation_date)>,
 <Feature: WEEKDAY(Creation_date)>,
 <Feature: YEAR(Creation_date)>,
 <Feature: MAX(Orders.COUNT(Payments))>,
 <Feature: MAX(Orders.MEAN(Payments.Price))>,
 <Feature: MAX(Orders.MIN(Payments.Price))>,
 <Feature: MAX(Orders.SKEW(Payments.Price))>,
 <Feature: MAX(Orders.STD(Payments.Price))>,
 <Feature: MAX(Orders.SUM(Payments.Price))>,
 <Feature: MEAN(Orders.COUNT(Payments))>,
 <Feature: MEAN(Orders.MAX(Payments.Price))>,
 <Feature: MEAN(Orders.MEAN(Payments.Price))>,
 <Feature: MEAN(Orders.MIN(Payments.Price))>,
 <Feature: MEAN(Orders.SKEW(Payments.Price))>,
 <Feature: MEAN(Orders.STD(Payments.Price))>,
 <Feature: MEAN(Orders.SUM(Payments.Price))>,
 <Feature: MIN(Orders